# Basic LLM usage

Draive framework provides various ways to use LLM depending on the use case. The simplest interface is to generate text by using `generate_text` function. We can use it to make a simple text completion function.

In [1]:
from draive import generate_text


async def text_completion(text: str) -> str:
    # generate_text is a simple interface for generating text
    return await generate_text(
        # We have to provide instructions / system prompt to instruct the model
        instruction="Prepare the simplest completion of a given text",
        # input is provided separately
        input=text,
    )

The result of this function is a completion from a currently used model. What is a currently used model? We have to define it yet by providing basic setup of state and dependencies. In this example we are going to use OpenAI client, you have to provide the api key to that service in .env file with `OPENAI_API_KEY` key before running.

In [2]:
from draive import load_env

load_env()  # load .env variables

When we have .env loaded we can prepare a context scope with OpenAI client and use our function. Lowest level interface is called LMM since draive supports multi-model solutions out of the box. Assigning `invocation` to `openai_lmm_invocation` and using it as the context scope state selects this provider for our completions. We have also define OpenAI client dependency which defines functions for accessing OpenAI services.

In [3]:
from draive import LMM, OpenAIClient, ctx, openai_lmm_invocation

async with ctx.new(  # prepare new context
    "text_completion",
    state=[LMM(invocation=openai_lmm_invocation)],  # set currently used LMM to OpenAI
    dependencies=[OpenAIClient],  # use OpenAIClient dependency for accessing OpenAI services
):
    result: str = await text_completion(
        text="Roses are red...",
    )

    print(result)

Violets are blue


As we now know how to setup OpenAI as out LLM provider. We can start customizing it more by providing GPT model configuration.

In [4]:
from draive import OpenAIChatConfig

async with ctx.new(  # prepare the new context
    "text_completion",
    state=[
        LMM(invocation=openai_lmm_invocation),
        # define GPT model configuration as a context scope state
        OpenAIChatConfig(
            model="gpt-3.5-turbo",
            temperature=0.4,
        ),
    ],
    dependencies=[OpenAIClient],
):
    # now we are using gpt-3.5-turbo with temperature of 0.4
    result: str = await text_completion(
        text="Roses are red...",
    )

    print("RESULT GPT 3.5 | temperature 0.4:", result)

    # we can update the configuration to change any parameter for nested context
    with ctx.updated(
        # we are updating the current context value instead of making a new one
        # this allows to preserve other elements of the configuration
        ctx.state(OpenAIChatConfig).updated(
            model="gpt-4o",
        ),
    ):
        # now we are using gpt-4o with temperature of 0.4
        result = await text_completion(
            text="Roses are red...",
        )

        print("RESULT GPT 4o | temperature 0.4:", result)

    # we can also update the configuration for a single call
    # when using generate_text function directly
    # here we are using gpt-3.5-turbo with temperature of 0.7
    result = await generate_text(
        instruction="Prepare simplest completion of given text",
        input="Roses are red...",
        temperature=0.7,
    )

    print("RESULT GPT 3.5 | temperature 0.7:", result)

RESULT GPT 3.5 | temperature 0.4: Violets are blue.
RESULT GPT 4o | temperature 0.4: Violets are blue,
Sugar is sweet,
And so are you.
RESULT GPT 3.5 | temperature 0.7: Violets are blue.


Since we knows the basics, now we can examine the details of our execution to see what actually happened inside. We can setup the logger before execution to see context metrics logs.

In [5]:
from draive import setup_logging

setup_logging("text_completion")  # setup logger

async with ctx.new(  # prepare the context and see the execution metrics report
    "text_completion",
    state=[
        LMM(invocation=openai_lmm_invocation),
        OpenAIChatConfig(  # define GPT model configuration
            model="gpt-3.5-turbo",
            temperature=0.4,
        ),
    ],
    dependencies=[OpenAIClient],
):
    await text_completion(
        text="Roses are red...",
    )

    with ctx.updated(
        ctx.state(OpenAIChatConfig).updated(
            model="gpt-4o",
        ),
    ):
        await text_completion(
            text="Roses are red...",
        )

    await generate_text(
        instruction="Prepare simplest completion of given text",
        input="Roses are red...",
        temperature=0.7,
    )

07/Jun/2024:12:31:12 +0000 [INFO] [text_completion] [53d719cf34ec44c9bb7c9c2c73ac97a1|text_completion] started...
07/Jun/2024:12:31:12 +0000 [INFO] [text_completion] [53d719cf34ec44c9bb7c9c2c73ac97a1|lmm_generate_text] started...
07/Jun/2024:12:31:12 +0000 [INFO] [text_completion] [53d719cf34ec44c9bb7c9c2c73ac97a1|openai_lmm_completion] started...
07/Jun/2024:12:31:12 +0000 [DEBUG] [text_completion] [53d719cf34ec44c9bb7c9c2c73ac97a1|openai_lmm_completion] Requested OpenAI lmm
07/Jun/2024:12:31:13 +0000 [INFO] [text_completion] [53d719cf34ec44c9bb7c9c2c73ac97a1|openai_lmm_completion] ...finished after 0.62s
07/Jun/2024:12:31:13 +0000 [DEBUG] [text_completion] [53d719cf34ec44c9bb7c9c2c73ac97a1|lmm_generate_text] Received text generation result
07/Jun/2024:12:31:13 +0000 [INFO] [text_completion] [53d719cf34ec44c9bb7c9c2c73ac97a1|lmm_generate_text] ...finished after 0.62s
07/Jun/2024:12:31:13 +0000 [INFO] [text_completion] [53d719cf34ec44c9bb7c9c2c73ac97a1|lmm_generate_text] started...
07/

The more advanced usage and use cases can be explored in other notebooks.